<font color="blue" size="6"><b>키워드 정리 Test</b></font>


## API 키워드 전처리

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/크롤링/키워드 분석

/content/drive/MyDrive/크롤링/키워드 분석


In [ ]:
!pwd

/content/drive/MyDrive/크롤링/키워드 분석


In [ ]:
!ls

Category0_search.csv   pandas_crawling.ipynb  signaturehelper.py
Category0_search.xlsx  __pycache__	      테스트.py


In [ ]:
category1 = pd.read_excel("/content/drive/MyDrive/크롤링/키워드 분석/Category0_search.xlsx",sheet_name="카테고리 항목들 수집",header=None)

In [ ]:
category1.to_csv("/content/drive/MyDrive/크롤링/키워드 분석/Category0_search.csv")

In [ ]:
category1 = pd.read_csv("/content/drive/MyDrive/크롤링/키워드 분석/Category0_search.csv").T
category1.drop("Unnamed: 0", axis = 0 ,inplace =True)

In [ ]:
lsts = category1.iloc[0].to_numpy()  # Series를 numpy array 변환
fixed_len = 5-len(lsts) % 5 # 5열을 맞추기 위해 남는 수 유추
lsts = np.pad(lsts,(0,fixed_len),'constant',constant_values=None) # API 키워드 넣을때 ? : 5 shape 맞추기 위한 전처리
new_lsts = lsts.reshape(-1,5)
new_lsts

array([['니트/스웨터', '카디건', '원피스', '티셔츠', '블라우스/셔츠'],
       ['점퍼', '재킷', '코트', '바지', '청바지'],
       ['스커트', '레깅스', '트레이닝복', '조끼', '정장세트'],
       ['한복', '유니폼/단체복', '파티복', '레인코트', '점프슈트'],
       ['코디세트', '브라', '팬티', '브라팬티세트', '잠옷/홈웨어'],
       ['시즌성내의', '보정속옷', '슬립', '러닝/캐미솔', '속치마/속바지'],
       ['언더웨어소품', '니트/스웨터', '티셔츠', '셔츠/남방', '카디건'],
       ['점퍼', '재킷', '코트', '청바지', '바지'],
       ['조끼', '정장세트', '트레이닝복', '한복', '유니폼/단체복'],
       ['레인코트', '코디세트', '점프슈트', '팬티', '러닝'],
       ['러닝팬티세트', '잠옷/홈웨어', '시즌성내의', '보정속옷', None]], dtype=object)

In [ ]:
new_lsts.shape

(11, 5)

## 네이버 API

In [ ]:
# test cell
import time
import requests
import signaturehelper

def get_header(method, uri, api_key, secret_key, customer_id):
    timestamp = str(round(time.time() * 1000))
    signature = signaturehelper.Signature.generate(timestamp, method, uri, SECRET_KEY)
    return {'Content-Type': 'application/json; charset=UTF-8',
            'X-Timestamp': timestamp,
            'X-API-KEY': API_KEY,
            'X-Customer': str(CUSTOMER_ID),
            'X-Signature': signature}

API_KEY = '01000000004031c42277f90c7fdcbd2130a4916b0aecf181b89b3543abab0f90b463c42821'
SECRET_KEY = 'AQAAAABAMcQid/kMf9y9ITCkkWsKnGbFkehYuYSpmb3ygOH/tg=='
CUSTOMER_ID = '3003237'
BASE_URL =  'https://api.naver.com'

uri = '/keywordstool'
method = 'GET'
query = {
    "siteId" : "",
    "biztpId" : "",
    "hintKeywords":   new_lsts[0],
    "event": "",
    "month": "",
    "showDetail": "1"
}
response = requests.get(BASE_URL + uri ,
                        params=query,
                        headers=get_header(
                        method=method,
                        uri=uri,
                        api_key= API_KEY,
                        secret_key= SECRET_KEY,
                        customer_id= CUSTOMER_ID))
r_data = response.json()
keywordList = r_data['keywordList']
main_keywordList = keywordList[:5]

for idx,key in enumerate(main_keywordList):
    pcCnt = key["monthlyPcQcCnt"]
    mobileCnt = key["monthlyMobileQcCnt"]

    print(f"Pc 검색량 : {pcCnt}  \t  Mobile 검색량 : {mobileCnt}")



Pc 검색량 : 5900  	  Mobile 검색량 : 31500
Pc 검색량 : 770  	  Mobile 검색량 : 2760
Pc 검색량 : 77300  	  Mobile 검색량 : 480100
Pc 검색량 : < 10  	  Mobile 검색량 : < 10
Pc 검색량 : < 10  	  Mobile 검색량 : 40


In [ ]:
# 키워드 분석_version(1.0.2)
import time
import requests
import signaturehelper
from bs4 import BeautifulSoup
import openpyxl


def get_header(method, uri, api_key, secret_key, customer_id):
    timestamp = str(round(time.time() * 1000))
    signature = signaturehelper.Signature.generate(timestamp, method, uri, SECRET_KEY)
    return {'Content-Type': 'application/json; charset=UTF-8',
            'X-Timestamp': timestamp,
            'X-API-KEY': API_KEY,
            'X-Customer': str(CUSTOMER_ID),
            'X-Signature': signature}

API_KEY = '01000000004031c42277f90c7fdcbd2130a4916b0aecf181b89b3543abab0f90b463c42821'
SECRET_KEY = 'AQAAAABAMcQid/kMf9y9ITCkkWsKnGbFkehYuYSpmb3ygOH/tg=='
CUSTOMER_ID = '3003237'
BASE_URL =  'https://api.naver.com'

cnt = 0
n_cnt = 0

# 네이버 API
for idx,new_lst in enumerate(new_lsts):
    uri = '/keywordstool'
    method = 'GET'
    query = {
        "siteId" : "",
        "biztpId" : "",
        "hintKeywords": new_lst,
        "event": "",
        "month": "",
        "showDetail": "1"
    }
    response_api = requests.get(BASE_URL + uri ,
                            params=query,
                            headers=get_header(
                                method=method,
                                uri=uri,
                                api_key= API_KEY,
                                secret_key= SECRET_KEY,
                                customer_id= CUSTOMER_ID))
    r_data = response_api.json()
    keywordList = r_data['keywordList']
    main_keywordList = keywordList[:5]

    for idx,key in enumerate(main_keywordList):

        # 쇼핑몰 스토어 수
        url = f"https://search.shopping.naver.com/search/all?query={key['relKeyword']}%20%20%09%20%20&frm=NVSHSRC&vertical=search"
        response_shopping = requests.get(url, headers={"User-Agent":"Mozilla/5.0"})
        html = response_shopping.text
        soup = BeautifulSoup(html,"html.parser")
        total_num = soup.select(".subFilter_num__S9sle")[0].text.replace(",","")

        try :
            pcCnt = float(key["monthlyPcQcCnt"])
            mobileCnt = float(key["monthlyMobileQcCnt"])
            totalCnt = pcCnt + mobileCnt
            print(f"관련 키워드 : {key['relKeyword']}  \t  총 검색량 : {totalCnt} \t  총 스토어 수 {total_num}")
            cnt += 1
        except :
            print("검색량 10개 이하")
            n_cnt += 1

print(f"count : {cnt}  |  Uncount : {n_cnt}")


# print("관련 키: ", keywordList[0]['relKeyword'] )
# print("월간 검색수_PC: ",keywordList[0]['monthlyPcQcCnt'] )
# print("월간 검색수_Mobile: ",keywordList[0]['monthlyMobileQcCnt'] )
# print("월 평균 클릭수_PC: ",keywordList[0]['monthlyAvePcClkCnt'] )
# print("월 평균 클릭수_Mobile",keywordList[0]['monthlyAveMobileClkCnt'] )
# print("월 평균 클릭률_PC: ",keywordList[0]['monthlyAvePcCtr'] )
# print("월 평균 클릭수_Mobile: ",keywordList[0]['monthlyAveMobileCtr'] )
# print("경쟁정도: ",keywordList[0]['plAvgDepth'] )
# print("월평균 노출 광고수: ",keywordList[0]['compIdx'] )

관련 키워드 : 티셔츠  	  총 검색량 : 37780.0 	  총 스토어 수 66293255
관련 키워드 : 카디건  	  총 검색량 : 3120.0 	  총 스토어 수 9458468
관련 키워드 : 원피스  	  총 검색량 : 556000.0 	  총 스토어 수 28541537
검색량 10개 이하
검색량 10개 이하
관련 키워드 : 청바지  	  총 검색량 : 64490.0 	  총 스토어 수 12880245
관련 키워드 : 바지  	  총 검색량 : 27960.0 	  총 스토어 수 46657290
관련 키워드 : 재킷  	  총 검색량 : 3300.0 	  총 스토어 수 25334738
관련 키워드 : 코트  	  총 검색량 : 20930.0 	  총 스토어 수 10743296
관련 키워드 : 점퍼  	  총 검색량 : 14510.0 	  총 스토어 수 10321338
관련 키워드 : 레깅스  	  총 검색량 : 35050.0 	  총 스토어 수 4532565
관련 키워드 : 트레이닝복  	  총 검색량 : 20120.0 	  총 스토어 수 10235351
관련 키워드 : 조끼  	  총 검색량 : 23570.0 	  총 스토어 수 11694168
관련 키워드 : 스커트  	  총 검색량 : 26150.0 	  총 스토어 수 16139946
관련 키워드 : 정장세트  	  총 검색량 : 2320.0 	  총 스토어 수 2160274
관련 키워드 : 파티복  	  총 검색량 : 1520.0 	  총 스토어 수 2246040
관련 키워드 : 한복  	  총 검색량 : 76700.0 	  총 스토어 수 1036894
관련 키워드 : 레인코트  	  총 검색량 : 11200.0 	  총 스토어 수 1680225
관련 키워드 : 점프슈트  	  총 검색량 : 14150.0 	  총 스토어 수 1973925
검색량 10개 이하
관련 키워드 : 브라  	  총 검색량 : 31700.0 	  총 스토어 수 7416906
관련 키워드 : 브라팬티세트  	  총 검색량 

In [ ]:
# 키워드 분석_version(1.0.3)
import time
import requests
import signaturehelper
from bs4 import BeautifulSoup
import openpyxl
from tqdm import tqdm


def get_header(method, uri, api_key, secret_key, customer_id):
    timestamp = str(round(time.time() * 1000))
    signature = signaturehelper.Signature.generate(timestamp, method, uri, SECRET_KEY)
    return {'Content-Type': 'application/json; charset=UTF-8',
            'X-Timestamp': timestamp,
            'X-API-KEY': API_KEY,
            'X-Customer': str(CUSTOMER_ID),
            'X-Signature': signature}

API_KEY = '01000000004031c42277f90c7fdcbd2130a4916b0aecf181b89b3543abab0f90b463c42821'
SECRET_KEY = 'AQAAAABAMcQid/kMf9y9ITCkkWsKnGbFkehYuYSpmb3ygOH/tg=='
CUSTOMER_ID = '3003237'
BASE_URL =  'https://api.naver.com'

cnt = 0
n_cnt = 0

# 네이버 API

analysis_data = pd.DataFrame(columns=["relative_keyword","total_search","total_store_count","competitive_rate"])
for idx,new_lst in tqdm(enumerate(new_lsts)):
    uri = '/keywordstool'
    method = 'GET'
    query = {
        "siteId" : "",
        "biztpId" : "",
        "hintKeywords": new_lst,
        "event": "",
        "month": "",
        "showDetail": "1"
    }
    response_api = requests.get(BASE_URL + uri ,
                            params=query,
                            headers=get_header(
                                method=method,
                                uri=uri,
                                api_key= API_KEY,
                                secret_key= SECRET_KEY,
                                customer_id= CUSTOMER_ID))
    r_data = response_api.json()
    keywordList = r_data['keywordList']
    main_keywordList = keywordList[:5]

    for idx,key in enumerate(main_keywordList):

        # 쇼핑몰 스토어 수
        url = f"https://search.shopping.naver.com/search/all?query={key['relKeyword']}%20%20%09%20%20&frm=NVSHSRC&vertical=search"
        response_shopping = requests.get(url, headers={"User-Agent":"Mozilla/5.0"})
        html = response_shopping.text
        soup = BeautifulSoup(html,"html.parser")
        total_num = float(soup.select(".subFilter_num__S9sle")[0].text.replace(",",""))

        try :
            pcCnt = float(key["monthlyPcQcCnt"])
            mobileCnt = float(key["monthlyMobileQcCnt"])
            totalCnt = pcCnt + mobileCnt
            competive_rate = np.around(total_num/totalCnt,2)

            # DataFrame 형태 만들기
            tmp = pd.DataFrame({"relative_keyword":[key['relKeyword']],"total_search":[totalCnt],"total_store_count":[total_num],"competitive_rate":[competive_rate]})
            analysis_data = pd.concat([analysis_data,tmp],axis=0)
            # print(f"관련 키워드 : {key['relKeyword']}  \t  총 검색량 : {totalCnt} \t  총 스토어 수 {total_num}")
            cnt += 1
        except :
            n_cnt += 1

print(f"\ncount : {cnt}  |  Uncount : {n_cnt}")


# print("관련 키: ", keywordList[0]['relKeyword'] )
# print("월간 검색수_PC: ",keywordList[0]['monthlyPcQcCnt'] )
# print("월간 검색수_Mobile: ",keywordList[0]['monthlyMobileQcCnt'] )
# print("월 평균 클릭수_PC: ",keywordList[0]['monthlyAvePcClkCnt'] )
# print("월 평균 클릭수_Mobile",keywordList[0]['monthlyAveMobileClkCnt'] )
# print("월 평균 클릭률_PC: ",keywordList[0]['monthlyAvePcCtr'] )
# print("월 평균 클릭수_Mobile: ",keywordList[0]['monthlyAveMobileCtr'] )
# print("경쟁정도: ",keywordList[0]['plAvgDepth'] )
# print("월평균 노출 광고수: ",keywordList[0]['compIdx'] )

11it [02:09, 11.74s/it]

count : 41  |  Uncount : 14


In [ ]:
analysis_data.index = np.arange(1,analysis_data.shape[0]+1)
analysis_data

,relative_keyword,total_search,total_store_count,competitive_rate
1,티셔츠,37780.0,66293532.0,1754.73
2,카디건,3120.0,9458468.0,3031.56
3,원피스,556000.0,28541777.0,51.33
4,청바지,64490.0,12880442.0,199.73
5,바지,27960.0,46657487.0,1668.72
6,재킷,3300.0,25335671.0,7677.48
7,코트,20930.0,10743624.0,513.31
8,점퍼,14510.0,10321825.0,711.36
9,레깅스,35050.0,4532573.0,129.32
10,트레이닝복,20120.0,10235560.0,508.73


<font color="red" size="6"><b>대표 키워드 정리</b></font>


## 모든 category 엑셀 합치고 전처리

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%ls

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 7089-DD61

 C:\Users\kcjer\개인프로젝트\스마트스토어 디렉터리

2023-10-14  오전 09:52    <DIR>          .
2023-10-13  오후 10:31    <DIR>          ..
2023-10-14  오전 09:22    <DIR>          .ipynb_checkpoints
2023-10-14  오전 09:52           641,604 경쟁률분석_네이버api.ipynb
2023-09-24  오후 11:02             5,106 데이터랩_크롤링.py
2023-10-13  오후 10:34    <DIR>          데이터폴더
2023-09-25  오전 08:43    <DIR>          카테고리csv
               2개 파일             646,710 바이트
               5개 디렉터리  73,952,677,888 바이트 남음


In [4]:
# category 모두 불러오고 csv 파일 만들기
for i in range(12):
  #globals()는 파이썬 내장 함수로, 현재 전역 심볼 테이블을 나타내는 딕셔너리를 반환
  globals()[f"category{i}"] = pd.read_excel(f"/content/drive/MyDrive/크롤링/네이버쇼핑몰크롤링/Category{i}_search.xlsx",sheet_name="카테고리 항목들 수집",header=None)
  globals()[f"category{i}"].to_csv(f"/content/drive/MyDrive/크롤링/네이버쇼핑몰크롤링/Category{i}_search.csv")

NameError: name 'pd' is not defined

In [ ]:
# category들 전처리
for i in range(12):
  globals()[f"category{i}"] = pd.read_csv(f"/content/drive/MyDrive/크롤링/네이버쇼핑몰크롤링/Category{i}_search.csv").T
  globals()[f"category{i}"].drop("Unnamed: 0", axis = 0 ,inplace =True)

In [ ]:
lsts = np.array([])
for i in range(12):
  tmp = globals()[f"category{i}"].iloc[0].to_numpy()  # Series를 numpy array 변환
  lsts = np.concatenate((lsts,tmp))

In [ ]:
# 띄어쓰기 and "/" 제외  -> 제외 안하면 검색 할 떄 invalid 뜸
fixed_lsts = []
for lst in lsts :
  items = lst.split('/')
  item  = [item.replace(" ","") for item in items]
  fixed_lsts.extend(item)
fixed_lsts = np.array(fixed_lsts)
fixed_lsts.shape


(2539,)

In [ ]:
fixed_len = 5-len(lsts) % 5 # 5열을 맞추기 위해 남는 수 유추
new_lsts = np.pad(fixed_lsts,(0,fixed_len),'constant',constant_values=None) # API 키워드 넣을때 ? : 5 shape 맞추기 위한 전처리
new_lsts.shape

(2540,)

In [ ]:
new_lsts = new_lsts.reshape(-1,5)
new_lsts.shape

(508, 5)

In [ ]:
pd.set_option('display.max_rows',None)
pd.DataFrame(new_lsts)

,0,1,2,3,4
0,니트,스웨터,카디건,원피스,티셔츠
1,블라우스,셔츠,점퍼,재킷,코트
2,바지,청바지,스커트,레깅스,트레이닝복
3,조끼,정장세트,한복,유니폼,단체복
4,파티복,레인코트,점프슈트,코디세트,브라
5,팬티,브라팬티세트,잠옷,홈웨어,시즌성내의
6,보정속옷,슬립,러닝,캐미솔,속치마
7,속바지,언더웨어소품,니트,스웨터,티셔츠
8,셔츠,남방,카디건,점퍼,재킷
9,코트,청바지,바지,조끼,정장세트


In [ ]:
# 키워드 분석_version(1.2.0)
import time
import requests
import signaturehelper
from bs4 import BeautifulSoup
import openpyxl
from tqdm import tqdm


def get_header(method, uri, api_key, secret_key, customer_id):
    timestamp = str(round(time.time() * 1000))
    signature = signaturehelper.Signature.generate(timestamp, method, uri, SECRET_KEY)
    return {'Content-Type': 'application/json; charset=UTF-8',
            'X-Timestamp': timestamp,
            'X-API-KEY': API_KEY,
            'X-Customer': str(CUSTOMER_ID),
            'X-Signature': signature}

API_KEY = '01000000004031c42277f90c7fdcbd2130a4916b0aecf181b89b3543abab0f90b463c42821'
SECRET_KEY = 'AQAAAABAMcQid/kMf9y9ITCkkWsKnGbFkehYuYSpmb3ygOH/tg=='
CUSTOMER_ID = '3003237'
BASE_URL =  'https://api.naver.com'

cnt = 0
n_cnt = 0

# 네이버 API

analysis_data = pd.DataFrame(columns=["relative_keyword","total_search","total_store_count","competitive_rate"])
for idx, new_lst in enumerate(new_lsts):
    print(f"\n{idx} : ",end=" ")
    uri = '/keywordstool'
    method = 'GET'
    query = {
        "siteId" : "",
        "biztpId" : "",
        "hintKeywords": new_lst,
        "event": "",
        "month": "",
        "showDetail": "1"
    }
    response_api = requests.get(BASE_URL + uri ,
                            params=query,
                            headers=get_header(
                                method=method,
                                uri=uri,
                                api_key= API_KEY,
                                secret_key= SECRET_KEY,
                                customer_id= CUSTOMER_ID))
    try:   # 처음 try는 키워드의 invaild 또는 스토어 수 requests 에 실패 대응
      r_data = response_api.json()
      keywordList = r_data['keywordList']
      main_keywordList = keywordList[:5]

      for key_idx,key in enumerate(main_keywordList):

          # 쇼핑몰 스토어 수
          url = f"https://search.shopping.naver.com/search/all?query={key['relKeyword']}%20%20%09%20%20&frm=NVSHSRC&vertical=search"
          response_shopping = requests.get(url, headers={"User-Agent":"Mozilla/5.0"})
          html = response_shopping.text
          soup = BeautifulSoup(html,"html.parser")
          total_num = float(soup.select(".subFilter_num__S9sle")[0].text.replace(",",""))

          try :
              pcCnt = float(key["monthlyPcQcCnt"])
              mobileCnt = float(key["monthlyMobileQcCnt"])
              totalCnt = pcCnt + mobileCnt
              competive_rate = np.around(total_num/totalCnt,2)

              # DataFrame 형태 만들기
              tmp = pd.DataFrame({"relative_keyword":[key['relKeyword']],"total_search":[totalCnt],"total_store_count":[total_num],"competitive_rate":[competive_rate]})
              analysis_data = pd.concat([analysis_data,tmp],axis=0)
              # print(f"관련 키워드 : {key['relKeyword']}  \t  총 검색량 : {totalCnt} \t  총 스토어 수 {total_num}")
              cnt += 1
              print(".", end= "")
          except :
            # 검색량 10개 이하 경우 생략
              print(":", end= "")
              n_cnt += 1

    except :
      for i in range(key_idx,5) :  #  key_idx를 확인해서 문제 있는 부분 부터 다시 시도
        try :
          query = {
                "siteId" : "",
                "biztpId" : "",
                "hintKeywords":   new_lst[i],
                "event": "",
                "month": "",
                "showDetail": "1"}

          response = requests.get(BASE_URL + uri ,
                        params=query,
                        headers=get_header(
                        method=method,
                        uri=uri,
                        api_key= API_KEY,
                        secret_key= SECRET_KEY,
                        customer_id= CUSTOMER_ID))

          r_data = response.json()
          keywordList = r_data['keywordList']
          main_keywordList = keywordList[0]

          # 쇼핑몰 스토어 수
          url = f"https://search.shopping.naver.com/search/all?query={main_keywordList['relKeyword']}%20%20%09%20%20&frm=NVSHSRC&vertical=search"
          response_shopping = requests.get(url, headers={"User-Agent":"Mozilla/5.0"})
          html = response_shopping.text
          soup = BeautifulSoup(html,"html.parser")
          total_num = float(soup.select(".subFilter_num__S9sle")[0].text.replace(",",""))


          try :  # 문제있는 부분부터 다시 시작해서 10개 이하 필터링
            pcCnt = float(main_keywordList["monthlyPcQcCnt"])
            mobileCnt = float(main_keywordList["monthlyMobileQcCnt"])
            totalCnt = pcCnt + mobileCnt
            competive_rate = np.around(total_num/totalCnt,2)
            tmp = pd.DataFrame({"relative_keyword":[main_keywordList['relKeyword']],"total_search":[totalCnt],"total_store_count":[total_num],"competitive_rate":[competive_rate]})
            analysis_data = pd.concat([analysis_data,tmp],axis=0)
            cnt += 1
            print(",",end="")
          except :
              print(":",end="")
              n_cnt += 1

        except  IndexError as e:  # 문제 있는 부분은 쇼핑에 "도서" 부분으로 데이터 수집 제외
          print(" 도서 책 제외 ", end= "")
          n_cnt += 1

print(f"\nCOUNT : {cnt}  |  UNCOUNT : {n_cnt}")



0 :  .....
1 :  .....
2 :  .....
3 :  .....
4 :  .....
5 :  ....:
6 :  .....
7 :  ...:.
8 :  .....
9 :  .....
10 :  .....
11 :  ....:
12 :  ....:
13 :  .....
14 :  .....
15 :  .....
16 :  .....
17 :  .....
18 :  .....
19 :  ....:
20 :  .....
21 :  .....
22 :  .....
23 :  ....:
24 :  ...:.
25 :  ...::
26 :  .....
27 :  ....:
28 :  .....
29 :  .....
30 :  .....
31 :  .....
32 :  .....
33 :  .....
34 :  ..:..
35 :  .....
36 :  .....
37 :  .....
38 :  .:..:
39 :  .....
40 :  .....
41 :  .....
42 :  .....
43 :  .....
44 :  .....
45 :  ....:
46 :  .....
47 :  .....
48 :  ....:
49 :  .....
50 :  .....
51 :  .....
52 :  .....
53 :  .....
54 :  ...:.
55 :  .....
56 :  .....
57 :  .....
58 :  .....
59 :  .....
60 :  .....
61 :  .....
62 :  .....
63 :  .....
64 :  .....
65 :  .....
66 :  .....
67 :  ..::.
68 :  ...:.
69 :  .....
70 :  ...::
71 :  .....
72 :  ....:
73 :  ...::
74 :  ....:
75 :  ....:
76 :  ....:
77 :  .....
78 :  ..:..
79 :  .....
80 :  .....
81 :  ..:..
82 :  ....:
83 :  .....
8

In [ ]:
analysis_data.to_csv("대표카테고리_키워드정리.csv", index=False)

In [ ]:
analysis_data.index = np.arange(1,len(analysis_data)+1)
analysis_data.shape

(2260, 4)

In [ ]:
pd.set_option('display.max_row', None)
analysis_data[analysis_data["competitive_rate"]< 3]

,relative_keyword,total_search,total_store_count,competitive_rate
187,골드바,54720.0,117291.0,2.14
191,실버바,6170.0,7219.0,1.17
209,톤업크림,28440.0,68182.0,2.40
229,속눈썹영양제,46050.0,52519.0,1.14
273,남성청결제,11690.0,20050.0,1.72
377,MP3다운로드,2110.0,1722.0,0.82
440,김치냉장고,119700.0,220598.0,1.84
460,음식물처리기,134500.0,59651.0,0.44
482,탄산수제조기,11420.0,33819.0,2.96
483,업소용음식물처리기,3070.0,7088.0,2.31


#**테스트**

In [ ]:
new_lsts[442]

array(['장르소설', '테마문학', '한국소설', '비평', '창작'], dtype='<U12')

In [ ]:
# test cell
import time
import requests
import signaturehelper
from bs4 import BeautifulSoup
import openpyxl
from tqdm import tqdm


def get_header(method, uri, api_key, secret_key, customer_id):
    timestamp = str(round(time.time() * 1000))
    signature = signaturehelper.Signature.generate(timestamp, method, uri, SECRET_KEY)
    return {'Content-Type': 'application/json; charset=UTF-8',
            'X-Timestamp': timestamp,
            'X-API-KEY': API_KEY,
            'X-Customer': str(CUSTOMER_ID),
            'X-Signature': signature}

API_KEY = '01000000004031c42277f90c7fdcbd2130a4916b0aecf181b89b3543abab0f90b463c42821'
SECRET_KEY = 'AQAAAABAMcQid/kMf9y9ITCkkWsKnGbFkehYuYSpmb3ygOH/tg=='
CUSTOMER_ID = '3003237'
BASE_URL =  'https://api.searchad.naver.com'
problem = 441
main_keywordList = new_lsts[problem]

uri = '/keywordstool'
method = 'GET'
query = {
    "siteId" : "",
    "biztpId" : "",
    "hintKeywords":   main_keywordList,
    "event": "",
    "month": "",
    "showDetail": "1"
}
response = requests.get(BASE_URL + uri ,
                        params=query,
                        headers=get_header(
                        method=method,
                        uri=uri,
                        api_key= API_KEY,
                        secret_key= SECRET_KEY,
                        customer_id= CUSTOMER_ID))
try :
    r_data = response.json()

    keywordList = r_data['keywordList']
    main_keywordList = keywordList[:5]
    for idx,key in enumerate(main_keywordList):
        pcCnt = key["monthlyPcQcCnt"]
        mobileCnt = key["monthlyMobileQcCnt"]
        print(f"Pc 검색량 : {pcCnt}  \t  Mobile 검색량 : {mobileCnt}")
except:
    for j in range(5) :
      try :
        main_keywordList =  new_lsts[problem][j]
        query = {
              "siteId" : "",
              "biztpId" : "",
              "hintKeywords":   main_keywordList,
              "event": "",
              "month": "",
              "showDetail": "1"}
        response = requests.get(BASE_URL + uri ,
                        params=query,
                        headers=get_header(
                        method=method,
                        uri=uri,
                        api_key= API_KEY,
                        secret_key= SECRET_KEY,
                        customer_id= CUSTOMER_ID))
        r_data = response.json()

        keywordList = r_data['keywordList']
        main_keywordList = keywordList[0]


        pcCnt = main_keywordList["monthlyPcQcCnt"]
        mobileCnt = main_keywordList["monthlyMobileQcCnt"]
        print(f"Pc 검색량 : {pcCnt}  \t  Mobile 검색량 : {mobileCnt}")

      except  Exception as ex:
         print(ex)
         print('There are invalid parameters')








Pc 검색량 : 2070  	  Mobile 검색량 : 7200
Pc 검색량 : 3090  	  Mobile 검색량 : 9690
Pc 검색량 : 4670  	  Mobile 검색량 : 25300
Pc 검색량 : < 10  	  Mobile 검색량 : < 10
Pc 검색량 : 710  	  Mobile 검색량 : 2620


In [ ]:
tests = new_lsts[507]
tests

array(['EBS초등', '기타', '도서', '만화', 'None'], dtype='<U12')

In [ ]:
# 키워드 분석_version(1.2.0)
import time
import requests
import signaturehelper
from bs4 import BeautifulSoup
import openpyxl
from tqdm import tqdm


def get_header(method, uri, api_key, secret_key, customer_id):
    timestamp = str(round(time.time() * 1000))
    signature = signaturehelper.Signature.generate(timestamp, method, uri, SECRET_KEY)
    return {'Content-Type': 'application/json; charset=UTF-8',
            'X-Timestamp': timestamp,
            'X-API-KEY': API_KEY,
            'X-Customer': str(CUSTOMER_ID),
            'X-Signature': signature}

API_KEY = '01000000004031c42277f90c7fdcbd2130a4916b0aecf181b89b3543abab0f90b463c42821'
SECRET_KEY = 'AQAAAABAMcQid/kMf9y9ITCkkWsKnGbFkehYuYSpmb3ygOH/tg=='
CUSTOMER_ID = '3003237'
BASE_URL =  'https://api.naver.com'

cnt = 0
n_cnt = 0

# 네이버 API

analysis_data = pd.DataFrame(columns=["relative_keyword","total_search","total_store_count","competitive_rate"])
for idx, new_lst in enumerate(tests,441):  # ********여기 부분만 수정*****
    print(f"\n{idx} : ",end=" ")
    uri = '/keywordstool'
    method = 'GET'
    query = {
        "siteId" : "",
        "biztpId" : "",
        "hintKeywords": new_lst,
        "event": "",
        "month": "",
        "showDetail": "1"
    }
    response_api = requests.get(BASE_URL + uri ,
                            params=query,
                            headers=get_header(
                                method=method,
                                uri=uri,
                                api_key= API_KEY,
                                secret_key= SECRET_KEY,
                                customer_id= CUSTOMER_ID))
    try:   # 처음 try는 키워드의 invaild 또는 스토어 수 requests 에 실패 대응
      r_data = response_api.json()
      keywordList = r_data['keywordList']
      main_keywordList = keywordList[:5]

      for key_idx,key in enumerate(main_keywordList):

          # 쇼핑몰 스토어 수
          url = f"https://search.shopping.naver.com/search/all?query={key['relKeyword']}%20%20%09%20%20&frm=NVSHSRC&vertical=search"
          response_shopping = requests.get(url, headers={"User-Agent":"Mozilla/5.0"})
          html = response_shopping.text
          soup = BeautifulSoup(html,"html.parser")
          total_num = float(soup.select(".subFilter_num__S9sle")[0].text.replace(",",""))

          try :
              pcCnt = float(key["monthlyPcQcCnt"])
              mobileCnt = float(key["monthlyMobileQcCnt"])
              totalCnt = pcCnt + mobileCnt
              competive_rate = np.around(total_num/totalCnt,2)

              # DataFrame 형태 만들기
              tmp = pd.DataFrame({"relative_keyword":[key['relKeyword']],"total_search":[totalCnt],"total_store_count":[total_num],"competitive_rate":[competive_rate]})
              analysis_data = pd.concat([analysis_data,tmp],axis=0)
              # print(f"관련 키워드 : {key['relKeyword']}  \t  총 검색량 : {totalCnt} \t  총 스토어 수 {total_num}")
              cnt += 1
              print(".", end= "")
          except :
            # 검색량 10개 이하 경우 생략
              print(":", end= "")
              n_cnt += 1

    except :
      for i in range(key_idx,5) :  #  key_idx를 확인해서 문제 있는 부분 부터 다시 시도
        try :
          query = {
                "siteId" : "",
                "biztpId" : "",
                "hintKeywords":   new_lst[i],
                "event": "",
                "month": "",
                "showDetail": "1"}

          response = requests.get(BASE_URL + uri ,
                        params=query,
                        headers=get_header(
                        method=method,
                        uri=uri,
                        api_key= API_KEY,
                        secret_key= SECRET_KEY,
                        customer_id= CUSTOMER_ID))

          r_data = response.json()
          keywordList = r_data['keywordList']
          main_keywordList = keywordList[0]
          # 쇼핑몰 스토어 수
          url = f"https://search.shopping.naver.com/search/all?query={main_keywordList['relKeyword']}%20%20%09%20%20&frm=NVSHSRC&vertical=search"
          response_shopping = requests.get(url, headers={"User-Agent":"Mozilla/5.0"})
          html = response_shopping.text
          soup = BeautifulSoup(html,"html.parser")
          total_num = float(soup.select(".subFilter_num__S9sle")[0].text.replace(",",""))


          try :  # 문제있는 부분부터 다시 시작해서 10개 이하 필터링
            pcCnt = float(main_keywordList["monthlyPcQcCnt"])
            mobileCnt = float(main_keywordList["monthlyMobileQcCnt"])
            totalCnt = pcCnt + mobileCnt
            competive_rate = np.around(total_num/totalCnt,2)
            tmp = pd.DataFrame({"relative_keyword":[main_keywordList['relKeyword']],"total_search":[totalCnt],"total_store_count":[total_num],"competitive_rate":[competive_rate]})
            analysis_data = pd.concat([analysis_data,tmp],axis=0)
            cnt += 1
            print(",", end= "")
          except :
              print(":")
              n_cnt += 1

        except  IndexError as e:  # 문제 있는 부분은 쇼핑에 "도서" 부분으로 데이터 수집 제외
          print("\n도서 책 제외")
          n_cnt += 1


analysis_data.to_csv("대표카테고리_키워드정리.csv",index=False)
print(f"\nCOUNT : {cnt}  |  UNCOUNT : {n_cnt}")



441 :  ... , ,
442 :  
도서 책 제외
:

도서 책 제외

도서 책 제외
 ,
443 :   , , , ,
도서 책 제외

444 :  .....
445 :  . ,
도서 책 제외

도서 책 제외

도서 책 제외

446 :  . ,
도서 책 제외
 ,
도서 책 제외

447 :  . , ,
도서 책 제외

도서 책 제외

448 :  ... , ,
449 :  .....
450 :  .....
451 :  ... ,
도서 책 제외

452 :  ....
도서 책 제외

453 :  ..
도서 책 제외

도서 책 제외
 ,
454 :  .....
455 :  .
도서 책 제외
 , , ,
456 :  ...:.
457 :  .....
458 :  
도서 책 제외
 , , , ,
459 :  .....
460 :  .....
461 :  .....
462 :  .....
463 :  ....:
464 :  .....
465 :  .....
466 :  ...:.
467 :  .::..
468 :  .... ,
469 :  ..
도서 책 제외
 ,
도서 책 제외

470 :  .....
471 :  .....
472 :  .....
473 :  .....
474 :  ... ,
도서 책 제외

475 :  ..:..
476 :  .:..:
477 :  .....
478 :  ... ,
도서 책 제외

479 :  .:...
480 :  ... , ,
481 :  .....
482 :  ...
도서 책 제외
 ,
483 :  .....
484 :  ....:
485 :  ... , ,
486 :   , ,
도서 책 제외
 ,
도서 책 제외

487 :   , ,
도서 책 제외
 , ,
488 :  . ,
도서 책 제외
 ,
도서 책 제외

489 :  ...:
 ,
490 :  ...:.
491 :  .....
492 :  .....
493 :  .....
494 :  .....
495 :  .....
496 :  .....
497 :  ....

In [ ]:
# 키워드 분석_version(1.1.0)
import time
import requests
import signaturehelper
from bs4 import BeautifulSoup
import openpyxl
from tqdm import tqdm


def get_header(method, uri, api_key, secret_key, customer_id):
    timestamp = str(round(time.time() * 1000))
    signature = signaturehelper.Signature.generate(timestamp, method, uri, SECRET_KEY)
    return {'Content-Type': 'application/json; charset=UTF-8',
            'X-Timestamp': timestamp,
            'X-API-KEY': API_KEY,
            'X-Customer': str(CUSTOMER_ID),
            'X-Signature': signature}

API_KEY = '01000000004031c42277f90c7fdcbd2130a4916b0aecf181b89b3543abab0f90b463c42821'
SECRET_KEY = 'AQAAAABAMcQid/kMf9y9ITCkkWsKnGbFkehYuYSpmb3ygOH/tg=='
CUSTOMER_ID = '3003237'
BASE_URL =  'https://api.naver.com'

cnt = 0
n_cnt = 0

# 네이버 API

analysis_data = pd.DataFrame(columns=["relative_keyword","total_search","total_store_count","competitive_rate"])
for idx, new_lst in enumerate(tests,441):
    print(f"\n{idx} : ",end=" ")
    uri = '/keywordstool'
    method = 'GET'
    query = {
        "siteId" : "",
        "biztpId" : "",
        "hintKeywords": new_lst,
        "event": "",
        "month": "",
        "showDetail": "1"
    }
    response_api = requests.get(BASE_URL + uri ,
                            params=query,
                            headers=get_header(
                                method=method,
                                uri=uri,
                                api_key= API_KEY,
                                secret_key= SECRET_KEY,
                                customer_id= CUSTOMER_ID))
    try:   # 처음 try는 키워드의 invaild 또는 스토어 수 requests 에 실패 대응
      r_data = response_api.json()
      keywordList = r_data['keywordList']
      main_keywordList = keywordList[:5]

      for key_idx, key in enumerate(main_keywordList):
          print(key['relKeyword'], end= "")
          print(key_idx, end= "")
          # 쇼핑몰 스토어 수
          url = f"https://search.shopping.naver.com/search/all?query={key['relKeyword']}%20%20%09%20%20&frm=NVSHSRC&vertical=search"
          response_shopping = requests.get(url, headers={"User-Agent":"Mozilla/5.0"})
          html = response_shopping.text
          soup = BeautifulSoup(html,"html.parser")
          total_num = float(soup.select(".subFilter_num__S9sle")[0].text.replace(",",""))

          try :
              pcCnt = float(key["monthlyPcQcCnt"])
              mobileCnt = float(key["monthlyMobileQcCnt"])
              totalCnt = pcCnt + mobileCnt
              competive_rate = np.around(total_num/totalCnt,2)

              # DataFrame 형태 만들기
              tmp = pd.DataFrame({"relative_keyword":[key['relKeyword']],"total_search":[totalCnt],"total_store_count":[total_num],"competitive_rate":[competive_rate]})
              analysis_data = pd.concat([analysis_data,tmp],axis=0)
              # print(f"관련 키워드 : {key['relKeyword']}  \t  총 검색량 : {totalCnt} \t  총 스토어 수 {total_num}")
              cnt += 1
              print(".")
          except :
            # 검색량 10개 이하 경우 생략
              print(":")
              n_cnt += 1
    except :
      print('')
      for i in range(key_idx,5) :  #  key_idx를 확인해서 문제 있는 부분 부터 다시 시도
        try :
          query = {
                "siteId" : "",
                "biztpId" : "",
                "hintKeywords":   new_lst[i],
                "event": "",
                "month": "",
                "showDetail": "1"}

          response = requests.get(BASE_URL + uri ,
                        params=query,
                        headers=get_header(
                        method=method,
                        uri=uri,
                        api_key= API_KEY,
                        secret_key= SECRET_KEY,
                        customer_id= CUSTOMER_ID))

          r_data = response.json()
          keywordList = r_data['keywordList']
          main_keywordList = keywordList[0]

          print(main_keywordList["relKeyword"],end="")  # *변경 사항 *
          # 쇼핑몰 스토어 수  *변경된 부분 *
          url = f"https://search.shopping.naver.com/search/all?query={main_keywordList['relKeyword']}%20%20%09%20%20&frm=NVSHSRC&vertical=search"
          response_shopping = requests.get(url, headers={"User-Agent":"Mozilla/5.0"})
          html = response_shopping.text
          soup = BeautifulSoup(html,"html.parser")
          total_num = float(soup.select(".subFilter_num__S9sle")[0].text.replace(",",""))


          try :
            pcCnt = float(main_keywordList["monthlyPcQcCnt"])
            mobileCnt = float(main_keywordList["monthlyMobileQcCnt"])
            totalCnt = pcCnt + mobileCnt
            competive_rate = np.around(total_num/totalCnt,2)
            tmp = pd.DataFrame({"relative_keyword":[main_keywordList['relKeyword']],"total_search":[totalCnt],"total_store_count":[total_num],"competitive_rate":[competive_rate]})
            analysis_data = pd.concat([analysis_data,tmp],axis=0)
            cnt += 1
            print(" , ")
          except :
              print(":")
              n_cnt += 1

        except  IndexError as e:  # 문제 있는 부분은 쇼핑에 "도서" 부분으로 데이터 수집 제외
          print("  도서 책 제외")
          n_cnt += 1


analysis_data.to_csv("대표카테고리_키워드정리.csv",index=False)
print(f"\nCOUNT : {cnt}  |  UNCOUNT : {n_cnt}")




441 :  고전0.
문학1.
인삼2.
세계각국소설3
고전 , 
문학 , 

442 :  비평0
장르소설  도서 책 제외
테마문학:
한국소설  도서 책 제외
비평  도서 책 제외
창작 , 

COUNT : 6  |  UNCOUNT : 4


In [ ]:
# test cell
import time
import requests
import signaturehelper

def get_header(method, uri, api_key, secret_key, customer_id):
    timestamp = str(round(time.time() * 1000))
    signature = signaturehelper.Signature.generate(timestamp, method, uri, SECRET_KEY)
    return {'Content-Type': 'application/json; charset=UTF-8',
            'X-Timestamp': timestamp,
            'X-API-KEY': API_KEY,
            'X-Customer': str(CUSTOMER_ID),
            'X-Signature': signature}

API_KEY = '01000000004031c42277f90c7fdcbd2130a4916b0aecf181b89b3543abab0f90b463c42821'
SECRET_KEY = 'AQAAAABAMcQid/kMf9y9ITCkkWsKnGbFkehYuYSpmb3ygOH/tg=='
CUSTOMER_ID = '3003237'
BASE_URL =  'https://api.naver.com'

uri = '/keywordstool'
method = 'GET'
query = {
    "siteId" : "",
    "biztpId" : "",
    "hintKeywords":  "None",
    "event": "",
    "month": "",
    "showDetail": "1"
}
response = requests.get(BASE_URL + uri ,
                        params=query,
                        headers=get_header(
                        method=method,
                        uri=uri,
                        api_key= API_KEY,
                        secret_key= SECRET_KEY,
                        customer_id= CUSTOMER_ID))
r_data = response.json()
print(r_data)


{'keywordList': [{'relKeyword': 'NONE', 'monthlyPcQcCnt': 3220, 'monthlyMobileQcCnt': 7570, 'monthlyAvePcClkCnt': 0.0, 'monthlyAveMobileClkCnt': 2.0, 'monthlyAvePcCtr': 0.0, 'monthlyAveMobileCtr': 0.03, 'plAvgDepth': 3, 'compIdx': '높음'}]}


https://developers.naver.com/docs/serviceapi/datalab/shopping/shopping.md#%EC%87%BC%ED%95%91%EC%9D%B8%EC%82%AC%EC%9D%B4%ED%8A%B8-%EA%B0%9C%EC%9A%94

In [ ]:
for j in range(0,5) :
        try :
          query = {
                "siteId" : "",
                "biztpId" : "",
                "hintKeywords":   new_lst[j],
                "event": "",
                "month": "",
                "showDetail": "1"}

          response = requests.get(BASE_URL + uri ,
                        params=query,
                        headers=get_header(
                        method=method,
                        uri=uri,
                        api_key= API_KEY,
                        secret_key= SECRET_KEY,
                        customer_id= CUSTOMER_ID))

          r_data = response.json()
          keywordList = r_data['keywordList']
          main_keywordList = keywordList[0]
          print(main_keywordList)

          url = f"https://search.shopping.naver.com/search/all?query={main_keywordList['relKeyword']}%20%20%09%20%20&frm=NVSHSRC&vertical=search"
          response_shopping = requests.get(url, headers={"User-Agent":"Mozilla/5.0"})
          html = response_shopping.text
          soup = BeautifulSoup(html,"html.parser")
          total_num = float(soup.select(".subFilter_num__S9sle")[0].text.replace(",",""))


          try :
            pcCnt = float(main_keywordList["monthlyPcQcCnt"])
            mobileCnt = float(main_keywordList["monthlyMobileQcCnt"])
            totalCnt = pcCnt + mobileCnt
            competive_rate = np.around(total_num/totalCnt,2)
            tmp = pd.DataFrame({"relative_keyword":[main_keywordList['relKeyword']],"total_search":[totalCnt],"total_store_count":[total_num],"competitive_rate":[competive_rate]})
            analysis_data = pd.concat([analysis_data,tmp],axis=0)
            cnt += 1
            print(" , ")
          except :
              print(":")
              n_cnt += 1
        except  IndexError as e:
          print("  도서 책 제외")
          n_cnt += 1

{'relKeyword': '장르소설', 'monthlyPcQcCnt': 170, 'monthlyMobileQcCnt': 310, 'monthlyAvePcClkCnt': 0.6, 'monthlyAveMobileClkCnt': 3.4, 'monthlyAvePcCtr': 0.4, 'monthlyAveMobileCtr': 1.14, 'plAvgDepth': 9, 'compIdx': '중간'}
  도서 책 제외
{'relKeyword': '테마문학', 'monthlyPcQcCnt': '< 10', 'monthlyMobileQcCnt': '< 10', 'monthlyAvePcClkCnt': 0.0, 'monthlyAveMobileClkCnt': 0.0, 'monthlyAvePcCtr': 0.0, 'monthlyAveMobileCtr': 0.0, 'plAvgDepth': 0, 'compIdx': '낮음'}
:
{'relKeyword': '한국소설', 'monthlyPcQcCnt': 300, 'monthlyMobileQcCnt': 1080, 'monthlyAvePcClkCnt': 0.0, 'monthlyAveMobileClkCnt': 0.7, 'monthlyAvePcCtr': 0.0, 'monthlyAveMobileCtr': 0.07, 'plAvgDepth': 3, 'compIdx': '높음'}
  도서 책 제외
{'relKeyword': '비평', 'monthlyPcQcCnt': 2570, 'monthlyMobileQcCnt': 6590, 'monthlyAvePcClkCnt': 0.0, 'monthlyAveMobileClkCnt': 0.0, 'monthlyAvePcCtr': 0.0, 'monthlyAveMobileCtr': 0.0, 'plAvgDepth': 3, 'compIdx': '높음'}
  도서 책 제외
{'relKeyword': '창작', 'monthlyPcQcCnt': 920, 'monthlyMobileQcCnt': 1900, 'monthlyAvePcClkCnt